# Neural Network using PINN

The idea of this neural network is to take the following inputs:
   - dU/dy : Derivative of U in wall-normal direction
   - k : Turbulent kinetic energy, k = 0.5*(u'u' + v'v' + w'w') 
   - y^+ : Grid point in wall-normal direction
   - Re : Reynold value

and provide us the Reynolds Stress tensor components below:
   - u'u' : Variance of u
   - v'v' : Variance of v
   - w'w' : Variance of w
   - u'v' : Covariance of u and v 
   - u'w' : Covariance of u and w 
   - v'w' : Covariance of v and w

$$
\text{NN}\begin{bmatrix}
\frac{dU}{dy} \\
k \\
y^+ \\
Re
\end{bmatrix}
=
\begin{bmatrix}
u'u' \\
v'v' \\
w'w' \\
u'v' \\
u'w' \\
v'w'
\end{bmatrix}
$$

Additionnaly, the architecture that has been chosen for the model is the Residual Network !

## Neural Network Architecture

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np

class NN(nn.Module):
    def __init__(self, num_input, num_output, num_hidden, num_layers):
        super(NN, self).__init__()
        activation = nn.Tanh()
        self.input_layer = nn.Sequential(
            nn.Linear(num_input, num_hidden),
            activation
        )

        # Define hidden layers
        self.hidden_layers = nn.ModuleList()
        for _ in range(num_layers - 1):
            self.hidden_layers.append(nn.Sequential(
                nn.Linear(num_hidden, num_hidden),
                activation
            ))

        self.output_layer = nn.Linear(num_hidden, num_output)

    def forward(self, x):
        x = self.input_layer(x)
        for i, hidden_layer in enumerate(self.hidden_layers):
            if i % 2 == 0:  # Add residual connection every 2 layers
                x = x + hidden_layer(x)
            else:
                x = hidden_layer(x)
        x = self.output_layer(x)
        return x

## Defining the loss function

In [2]:
class Loss(nn.Module):
    def __init__(self, dns_rst, Lx, nu, y, add_dns_data):
        super(Loss, self).__init__()
        self.dns_rst = dns_rst
        self.Lx = Lx
        self.y = y
        self.nu = nu
        self.add_dns_data = add_dns_data

    def forward(self, predicted_reynolds_stress):
        # Extracting DNS data
        uu = self.dns_rst[:,0]
        vv = self.dns_rst[:,1]
        ww = self.dns_rst[:,2]
        uw = self.dns_rst[:,3]
        uv = self.dns_rst[:,4]
        vw = self.dns_rst[:,5]
        
        density = 1
        lambda_1 = 0.01
        lambda_2 = 0.001
        
        dns_reynolds_stress = torch.stack([uu, vv, ww, uw, uv, vw], dim=1)
        
        # Apply boundary conditions to the predicted Reynolds stress tensor
        predicted_reynolds_stress[0:3, :] = 0  # Set the first values of each column to zeros
        predicted_reynolds_stress[-3:, :] = 0
        
        dP_dx, d2U_d2y, d_uv_dy, dP_dy, d2_vv_d2y = compute_derivatives(self.Lx, self.y, predicted_reynolds_stress, self.add_dns_data)

        momentum_x_loss = torch.mean(torch.square((-((1/density) * dP_dx) +self.nu * d2U_d2y) - d_uv_dy)) # 

        momentum_y_loss = torch.mean(torch.square(-((1/density) * torch.tensor(dP_dy)) - torch.tensor(d2_vv_d2y)))
        
        loss = torch.mean((predicted_reynolds_stress - dns_reynolds_stress)**2)
        
        # Compute boundary condition penalties
        

        # Loss penalizing deviation from the boundary condition near the wall
    
        total_loss = loss + lambda_1 * momentum_x_loss + lambda_2 * momentum_y_loss 
        
        return total_loss
    
    # Put w'v' and v'w' equal to zero while preprocessing them ! Add BC (set every first values of components set to zero)
    

def compute_derivatives(Lx, y, predicted_stresses, add_dns_data):
    # x-direction
    dP_dx = add_dns_data[:, 1] / Lx
    dU_dy = np.gradient(add_dns_data[:, 0], y, edge_order=2)
    d2U_d2y = np.gradient(dU_dy, y, edge_order=2)
    d_uv_dy = np.gradient(predicted_stresses[:, 4].detach().numpy(), y, edge_order=2) 

    # y-direction
    dP_dy = np.gradient(add_dns_data[:, 1], y, edge_order=2)
    d_vv_dy = np.gradient(predicted_stresses[:, 1].detach().numpy(), y, edge_order=2)
    d2_vv_d2y = np.gradient(d_vv_dy, y, edge_order=2)

    return dP_dx, torch.tensor(d2U_d2y), torch.tensor(d_uv_dy), dP_dy, d2_vv_d2y

In [3]:
import pandas as pd
import numpy as np

# Couette section
file_paths = [r"C:\Users\Sohai\OneDrive - Cranfield University\GroupProject\Neural-Network\DNS_data\Couette_data\mirrored_LM_Couette_R0093_020PI_prof.csv",
             r"C:\Users\Sohai\OneDrive - Cranfield University\GroupProject\Neural-Network\DNS_data\Couette_data\mirrored_LM_Couette_R0093_100PI_prof.csv",
              r"C:\Users\Sohai\OneDrive - Cranfield University\GroupProject\Neural-Network\DNS_data\Couette_data\mirrored_LM_Couette_R0220_020PI_prof.csv",
              r"C:\Users\Sohai\OneDrive - Cranfield University\GroupProject\Neural-Network\DNS_data\Couette_data\mirrored_LM_Couette_R0220_100PI_prof.csv",
              r"C:\Users\Sohai\OneDrive - Cranfield University\GroupProject\Neural-Network\DNS_data\Couette_data\mirrored_LM_Couette_R0500_020PI_prof.csv"]


# Reynolds numbers
Re = [93, 93, 220, 220, 500]
Lx = [20 * np.pi, 100 * np.pi, 20 * np.pi, 100 * np.pi, 20 * np.pi]
nu = [6.66667e-04, 6.66667e-04, 2.50000e-04, 2.50000e-04, 1.00000e-04]

#Initialize the input list
input_features = []

# Initialize empty lists to store stress tensor components
uu_list = []
vv_list = []
ww_list = []
uw_list = []
uv_list = []
vw_list = []

u_list = []
k_list = []

# Loop through each file
for i, file_path in enumerate(file_paths):
    data = pd.read_csv(file_path)
    mean_velocity_grad = data['dU/dy'].values
    k = data['k'].values
    y_plus = data['y^+'].values
    P = data['P'].values
    U = data['U'].values

    # Combine all the input features needed for the model to use 
    features = np.column_stack((mean_velocity_grad, k, y_plus, np.full_like(U, Re[i]), U, P, np.full_like(U, Lx[i]), np.full_like(U, nu[i])))
    input_features.append(features)

X = np.concatenate(input_features, axis=0)

print("Shape of the input array:", X.shape)

for file_path in file_paths:
    # Read the CSV file
    data = pd.read_csv(file_path)
    # Extract the stress tensor columns and convert them to NumPy arrays
    uu = data["u'u'"].values
    vv = data["v'v'"].values
    ww = data["w'w'"].values
    uw = data["u'w'"].values
    uv = data["u'v'"].values
    vw = data["v'w'"].values
    
    uu_list.append(uu)
    vv_list.append(vv)
    ww_list.append(ww)
    uw_list.append(uw)
    uv_list.append(uv)
    vw_list.append(vw)

# Concatenate the stress tensor components along axis 0 to create the output labels
uu = np.concatenate(uu_list, axis=0)
vv = np.concatenate(vv_list, axis=0)
ww = np.concatenate(ww_list, axis=0)
uw = np.concatenate(uw_list, axis=0)
uv = np.concatenate(uv_list, axis=0)
vw = np.concatenate(vw_list, axis=0)

# Combine Reynolds stress tensor components
y = np.column_stack((uu, vv, ww, uw, uv, vw))

print("Shape of the output array:", y.shape)

# Convert the numpey array into tensor
X_train_tensor = torch.tensor(X, dtype=torch.float32)
y_train_tensor = torch.tensor(y, dtype=torch.float32)

print("Shape of the input tensor:", X.shape)
print("Shape of the output tensor:", y.shape)

Shape of the input array: (891, 8)
Shape of the output array: (891, 6)
Shape of the input tensor: (891, 8)
Shape of the output tensor: (891, 6)


## Train Section

In [4]:
# Initialization of the model
num_input = 4  # Number of input features
num_output = 6  #  Number of output features (for the 6 components of the Reynolds stress tensor)
num_hidden = 64  # Number of neurones
num_layers = 10 # Number of hidden layers

model = NN(num_input, num_output, num_hidden, num_layers)

# Defining the loss function
loss_fn = Loss(y_train_tensor, X_train_tensor[:,-2], X_train_tensor[:,-1], X_train_tensor[:, 2], X_train_tensor[:, 4:6])

# Defining the optimizer
optimizer = torch.optim.Adamax(model.parameters(), lr=0.00025, weight_decay=0.00001) 

# Training loop
num_epochs = 2000
for epoch in range(num_epochs):
    optimizer.zero_grad()  # Reset gradients
    
    predicted_reynolds_stress = model(X_train_tensor[:, :4])  
    
    # Add regularization term to the loss
    l2_regularization_loss = 0
    for param in model.parameters():
        l2_regularization_loss += torch.norm(param, p=2)  # L2 norm regularization
        
    loss = loss_fn(predicted_reynolds_stress)  # Compute loss
    
    lambda_reg = 0.0025
    loss += lambda_reg * l2_regularization_loss
    
    loss.backward()  # Backward pass
    
    optimizer.step()  # Update model parameters
    
    if (epoch) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.5f}')

print('Training finished!')

Epoch [1/2000], Loss: 6.19672
Epoch [101/2000], Loss: 1.41201
Epoch [201/2000], Loss: 0.99827
Epoch [301/2000], Loss: 0.58070
Epoch [401/2000], Loss: 0.36929
Epoch [501/2000], Loss: 0.30287
Epoch [601/2000], Loss: 0.23873
Epoch [701/2000], Loss: 0.21015
Epoch [801/2000], Loss: 0.19083
Epoch [901/2000], Loss: 0.17479
Epoch [1001/2000], Loss: 0.16290
Epoch [1101/2000], Loss: 0.15657
Epoch [1201/2000], Loss: 0.15201
Epoch [1301/2000], Loss: 0.14864
Epoch [1401/2000], Loss: 0.14571
Epoch [1501/2000], Loss: 0.14292
Epoch [1601/2000], Loss: 0.14065
Epoch [1701/2000], Loss: 0.13881
Epoch [1801/2000], Loss: 0.13713
Epoch [1901/2000], Loss: 0.13519
Training finished!


# Save Model

In [13]:
import torch

torch.save(model.state_dict(), 'model_RANS_based_PINN_couette.pth')


## Test Section

In [5]:
file_path = r"C:\Users\Sohai\OneDrive - Cranfield University\GroupProject\Neural-Network\DNS_data\Couette_data\mirrored_LM_Couette_R0500_100PI_prof.csv"

data = pd.read_csv(file_path)
# Extract the 'U' column and convert it to a NumPy array
mean_velocity_grad_test = data['dU/dy'].values
k_test = data['k'].values
y_plus_test = data['y^+'].values
Re_test = 500

# Combine all the input features needed for the model to use --- mean_velocity_grad_test    , np.full_like(U_test, 500)
features_test = np.column_stack((mean_velocity_grad_test, k_test, y_plus_test, np.full_like(mean_velocity_grad_test, Re_test)))

# Append the input features to the list
L_test = []
L_test.append(features_test)
X_test_ = np.concatenate(L_test, axis=0) 

print("Shape of X:", X_test_.shape)

# Initialize empty lists to store stress tensor components
uu_list = []
vv_list = []
ww_list = []
uw_list = []
uv_list = []
vw_list = []

data = pd.read_csv(file_path)

uu = data["u'u'"].values
vv = data["v'v'"].values
ww = data["w'w'"].values
uw = data["u'w'"].values
uv = data["u'v'"].values
vw = data["v'w'"].values

uu_list.append(uu)
vv_list.append(vv)
ww_list.append(ww)
uw_list.append(uw)
uv_list.append(uv)
vw_list.append(vw)

# Concatenate the stress tensor components along axis 0 to create the output columns
uu = np.concatenate(uu_list, axis=0)
vv = np.concatenate(vv_list, axis=0)
ww = np.concatenate(ww_list, axis=0)
uw = np.concatenate(uw_list, axis=0)
uv = np.concatenate(uv_list, axis=0)
vw = np.concatenate(vw_list, axis=0)

# Combine Reynolds stress tensor components
y_features_test = np.column_stack((uu, vv, ww, uw, uv, vw))

M = []
M.append(y_features_test)
y_test = np.concatenate(M, axis=0) 
print("Shape of y:", y_test.shape)

# Convert the numpey array into tensor
X_test_tensor = torch.tensor(X_test_, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

Shape of X: (255, 4)
Shape of y: (255, 6)


In [6]:
# Set the model to evaluation mode
model.eval()

with torch.no_grad():
    predicted_reynolds_stress = model(X_test_tensor[:, :4])

mse = nn.MSELoss()
test_loss = mse(predicted_reynolds_stress, y_test_tensor)

print("Mean Squared Error on Test Data:", test_loss.item())

Mean Squared Error on Test Data: 0.014313925057649612


# Calculate the Kinematic viscosity k

In [7]:
# Extract the columns of variances and calculate the kinematic viscosity:
selected_columns = predicted_reynolds_stress[:, [0, 1, 2]]
column_sum = torch.sum(selected_columns, dim=1)
k = 0.5 * column_sum

# Concatenate the column k with all the others
predicted_reynolds_stress_with_k = torch.cat((predicted_reynolds_stress, k.unsqueeze(1)), dim=1)


In [8]:
predicted_reynolds_stress_with_k.shape

torch.Size([255, 7])

# Save the predicted outcome 

In [10]:
tensor_array = predicted_reynolds_stress_with_k.numpy()

file_name = file_path.split('\\')[-1]
file_path = r"C:\Users\Sohai\OneDrive - Cranfield University\GroupProject\Neural-Network\Result_comparison\\" + "Result_RANS_" + file_name

# Header for the csv file
header = "u'u',v'v',w'w',u'w',u'v',v'w',k"

# Save the NumPy array to a CSV file with the header
np.savetxt(file_path, tensor_array, delimiter=',', header=header, comments='')
